# Setup

In [92]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, peak_widths 
import mplcursors
import bisect
from os import mkdir
from shapely.geometry import LineString
%matplotlib qt
import numpy as np

In [93]:
signals = {}
peaks = {}
peaks_w = {}
noise_floor = {}

In [94]:
def extract_data(name_diary,length=10):
    multipliers = np.ones((5,3))
    with open(name_diary) as file:
        d = file.readlines()
        i = 0
        j = 0
        while i < len(d):
            if('_ch' in d[i] or 'x:' in d[i]): 
                if 'e' not in d[i+1]:
                    d.insert(i+1, '\n')
                    d.insert(i+1, "1.0e+00 *\n")
                multipliers[(j//3)][j%3] = float(d[i+1][:-3])
                j += 1
            
            i+=1   
        #print(d) 
        #print(multipliers)
        d = "".join(d)
        d = d.split("ch1:")

    d = d[1:]
    for i, s in enumerate(d):
        d[i] = s.split('--')
        d[i] = d[i][0]
        d[i] = d[i].split('\n\n')
        #print(d)
        d[i] = [k for k in d[i] if len(k) > 100]
        d[i] = [a.split('\n') for a in d[i]]
        d[i] = [[n.replace(' ', '').replace('i', 'j') for n in j if len(n.replace(' ', '')) > 0 and 'x' not in n and 'ch' not in n] for j in d[i]]
        #print(len(d[i][0]))
        d[i] = [multipliers[i][k]*np.array([complex(n) if k < 2 else float(n) for n in j])[:100*length] for k, j in enumerate(d[i])]
    return d

In [95]:
def get_widths(x, y, peak_i):
    contour = y[peak_i]*0.5
    data = LineString(np.column_stack((x, y)))
    target = LineString(((x[0], contour), (x[-1], contour)))
    intercepts = data.intersection(target)
    cepts = []
    if intercepts.is_empty:
        return np.array([0, x[peak_i], x[peak_i]])
    
    if intercepts.geom_type == "Point":
        cepts = np.array([intercepts.x])
    else:
        cepts = np.array([i.x for i in intercepts.geoms])
    comps = np.sort(cepts-x[peak_i])
    #print(comps)
    right = bisect.bisect_right(comps,0)
    return np.array([contour, comps[right-1]+x[peak_i], comps[min(right, len(comps) - 1)]+x[peak_i]])

In [96]:
def plot_data(key, typ, run, rwidths_plots=[], width = False, combine = False):
    d = signals[key]
    if typ == "transfer":
        data = np.abs(d[run][1])/np.abs(d[run][0])
    elif typ == "out":
        data = np.abs(d[run][1])
    else:
        data = np.abs(d[run][0])
    try:
        mkdir(f"{key}")
    except:
        pass
    if(typ == "transfer"):
        plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
        plt.ylabel("Intensity (dB)")
    else:
        plt.plot(d[run][2], data, label=key, lw=0.75)
        plt.ylabel("Intensity (arbitrary units)")
    plt.xlabel("Frequency (Hz)")
    if(width):
        plt.hlines(*rwidths_plots, lw=3, label="width", color="C1")
    plt.title(f"{typ}, run={run}")
    '''
    Run 0 = sweep
    Run 1 = sine at 9.2Hz
    Run 2 = random
    Run 3 = sines at 9.2 and 13.6
    Run 4 = sines at 9.2 and 50
    '''
    plt.legend()
    #mplcursors.cursor(hover=True)
    plt.show()
    if not combine:
        plt.savefig(f"{key}/{typ}-{run}.pdf")
        #plt.pause(5)
        plt.close()
    else:
        mplcursors.cursor(hover=True)

In [184]:
def get_data(key,typ="transfer", run=0, plot=False, combine=False,length=10):
    '''
    Returns peaks, widths (6dB), plottable widths, and noise floor
    '''
    d = signals[key]
    if typ == "transfer":
        data = np.abs(d[run][1])/np.abs(d[run][0])
    elif typ == "out":
        data = np.abs(d[run][1])
    else:
        data = np.abs(d[run][0])
    prominences = np.linspace(0, 10000, int(1e8))[::-1]
    precise_pr = bisect.bisect_left(prominences, 3, key=lambda x: len(find_peaks(data, prominence=x,distance=3)[0]))
    #print(precise_height)
    rpeaks = find_peaks(data, prominence=prominences[precise_pr],distance=3*length)[0]
    #print(get_widths(d[0][2], data, rpeaks[0]))
    #temp = peak_widths(data, rpeaks, rel_height=0.5) 
    #rwidths, rwidths_plots = temp[0], temp[1:]
    #rwidths_plots = [20*np.log10(x) if i == 0 else x/10 for i,x in enumerate(rwidths_plots)]
    rwidths_plots = list(zip(*[get_widths(d[run][2], data, p) for p in rpeaks]))
    #print(rwidths_plots)
    #print(peak_widths(data, rpeaks, rel_height=0.5)[1:])
    if typ == "transfer":
        rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
    else:
        rwidths_plots = [np.array(x) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
    #print(rwidths_plots)
    rwidths = [rwidths_plots[2][i] -rwidths_plots[1][i] for i in range(len(rwidths_plots[0]))]
    if typ != "transfer":
        noise = np.std(data[length*40:length*60])/np.average(data[length*40:length*60])
    else:
        noise = np.std(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))/np.average(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))
    if(plot):
        plot_data(key, typ=typ, run=run, rwidths_plots=rwidths_plots, width=True,combine=combine)
    list_of_peaks = list(d[run][2][rpeaks])
    peak_heigths = list(data[rpeaks])
    if list_of_peaks[0] > 6:
        list_of_peaks.insert(0,0)
        peak_heigths.insert(0,0)
    if list_of_peaks[-1] > 15:
        list_of_peaks = list_of_peaks[:-1]
        peak_heigths = peak_heigths[:-1]
    if list_of_peaks[-1] < 11:
        list_of_peaks.append(0)
        peak_heigths.append(0)
    if len(peak_heigths) < 3:
        list_of_peaks.insert(1,0)
        peak_heigths.insert(1,0)
    if len(peak_heigths) != 3 and length == 10:
        print(d[run][2][rpeaks], data[rpeaks])
        raise Exception('AAAAAAAAAAA')
    return (list_of_peaks, peak_heigths), rwidths, rwidths_plots, noise

# Individual signals

## Kaiser

### Kaiser 1

In [98]:
key = "Kaiser (param 1)"
signals[key] = extract_data("code/myKaiser1Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]



C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.std(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))/np.average(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.std(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))/np.average(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))
C:\Users\gabri\AppData

### Kaiser 3

In [99]:
key = "Kaiser (param 3)"
signals[key] = extract_data("code/myKaiser3Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)


### Kaiser 9

In [100]:
key = "Kaiser (param 9)"
signals[key] = extract_data("code/myKaiser9Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:7: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.std(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))/np.average(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:4: RuntimeWarning: divide by z

## Flat-top

In [101]:
key = "Flat-top"
signals[key] = extract_data("code/myFlattopDiary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:7: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:4: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\292135

## Blackman

In [102]:
key = "Blackman"
signals[key] = extract_data("code/myBlackmanDiary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:7: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.std(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))/np.average(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:4: RuntimeWarning: divide by z

## Rect (10s)

In [103]:
key = "No windowing"
signals[key] = extract_data("code/myRectDiary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:7: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\creation.py:218: RuntimeWarning: invalid value encountered in linestrings
  return lib.linestrings(coords, np.intc(handle_nan), out=out, **kwargs)
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:4: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:7: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\

## Chebyshev

### Chebyshev 20

In [104]:
key = "Chebyshev (-20dB)"
signals[key] = extract_data("code/myCheb20Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]


### Chebyshev 60

In [105]:
key = "Chebyshev (-60dB)"
signals[key] = extract_data("code/myCheb60Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

### Chebyshev 100

In [106]:
key = "Chebyshev (-100dB)"
signals[key] = extract_data("code/myCheb100Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.std(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))/np.average(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\3346661131.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.std(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))/np.average(np.nan_to_num(np.abs(20*np.log10(data[length*40:length*60])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData

# Combined graphs

## Run 0 (sweep)

### Transfer function

In [90]:
for g in ["transfer"]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=0, combine=True, width=False)
    plt.savefig(f"Run 0 -- combined -- {g}.pdf")
    plt.close()

C:\Users\gabri\AppData\Local\Temp\ipykernel_12156\2921350288.py:4: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])


In [107]:
colourmap = plt.get_cmap("turbo")
data = np.array([noise_floor[s]["transfer"][0] for s in signals.keys()])
plt.bar(signals.keys(), data, color=colourmap((data-np.min(data))/(np.max(data)-np.min(data))))
plt.xticks(rotation=90)
plt.ylabel("Noise floor (dB)")
plt.title("Noise floor of output signal at sweep")
plt.savefig("noise_floor_transfer_run_0_out.pdf",bbox_inches='tight')

In [75]:
###WIDTHS
colourmap = plt.get_cmap("turbo")
data = [peaks_w[s]["transfer"][0][0] for s in signals.keys()]
max_L = 0
print(data)

x_poses = np.arange(len(data))
w = 0.3
aa = ["Peak 1","Peak 2","Peak 3"]
data = list(zip(*data))
print(data)
for i, peeks in enumerate(data[:3]):
    plt.bar(x_poses + i*w - w/2, peeks, width=w,label=f"Peak {i}")

plt.xticks(x_poses, signals.keys(), rotation=90)
plt.legend()
plt.ylabel("Peak width (Hz)")
plt.title("Peak width of transfer at sweep input")
plt.savefig("peak_width_transfer_run_0_out.pdf",bbox_inches='tight')

[[np.float64(0.1757277256688612), np.float64(0.13874933108493614), np.float64(0.19051854173343763)], [np.float64(0.11044996397779538), np.float64(0.33604796286074645), np.float64(0.19177973369960277)], [np.float64(0.12171162418574788), np.float64(0.6497019170745872)], [np.float64(0.1056010318715046)], [np.float64(0.1259250846158576), np.float64(0.11111824948750026), np.float64(0.6354929064367774)], [np.float64(0.1557188739355806), np.float64(0.12485075933361323), np.float64(0.2121497017827494)], [np.float64(0.2348795107588515), np.float64(0.18508349283525938)], [np.float64(0.1498948506882306), np.float64(0.12320835264035779), np.float64(0.318709545187863)], [np.float64(0.12545103482858888), np.float64(0.11644958002629835)], [np.float64(0.5416973923342159)]]
[(np.float64(0.1757277256688612), np.float64(0.11044996397779538), np.float64(0.12171162418574788), np.float64(0.1056010318715046), np.float64(0.1259250846158576), np.float64(0.1557188739355806), np.float64(0.2348795107588515), np.f

In [155]:
###FREQS
colourmap = plt.get_cmap("turbo")
data = [list(peaks[s]["transfer"][0][0]) for s in signals.keys()][:-1]
max_L = 0
print(data)

x_poses = np.arange(len(data[0]))
w = 0.08
aa = ["Peak 1","Peak 2","Peak 3"]
#data = list(zip(*data))
print(data)
for i, peeks in enumerate(data):
    plt.bar(x_poses + i*w - 3.5*w, peeks, width=w,label=f"{list(signals.keys())[:-1][i]}")

plt.xticks(x_poses, aa, rotation=90)
plt.legend()
plt.ylabel("Peak place (Hz)")
plt.title("Largest 3 peaks of transfer at sweep input")
plt.savefig("peak_places_transfer_run_0_out.pdf",bbox_inches='tight')

[[np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [np.float64(0.8003), np.float64(9.2032), np.float64(13.5046)], [0, np.float64(7.8027), np.float64(13.5046)], [0, np.float64(9.0031), 0], [np.float64(0.3001), np.float64(8.803), np.float64(13.5046)], [np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [0, np.float64(10.1035), np.float64(13.8047)], [np.float64(1.0003), np.float64(8.703), np.float64(13.4046)]]
[[np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [np.float64(0.8003), np.float64(9.2032), np.float64(13.5046)], [0, np.float64(7.8027), np.float64(13.5046)], [0, np.float64(9.0031), 0], [np.float64(0.3001), np.float64(8.803), np.float64(13.5046)], [np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [0, np.float64(10.1035), np.float64(13.8047)], [np.float64(1.0003), np.float64(8.703), np.float64(13.4046)]]


In [16]:
colourmap = plt.get_cmap("turbo")

data = np.array([sorted(peaks[s]["transfer"][0][1])[-1] for s in signals.keys()])

plt.bar(signals.keys(), data, color=colourmap(data/np.max(data)))
plt.xticks(rotation=90)
plt.ylabel("Main peak amplitude (dB)")
plt.title("Main peak amplitude of output signal at sweep input")
plt.savefig("main_peak_amplitude_transfer_run_0_out.pdf",bbox_inches='tight')

### Separate channels

#### Channel 1 (input force)

In [92]:
for g in ["out"]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=0, combine=True, width=False)
    plt.savefig(f"Run 0 -- combined -- {g}.pdf")
    plt.close()

#### Channel 2 (output)

In [91]:
for g in ["in"]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=0, combine=True, width=False)
    plt.savefig(f"Run 0 -- combined -- {g}.pdf")
    plt.close()

## Run 1 (pure resonance at 9.2 Hz)

In [26]:
for g in ["out","in"][::-1]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=1, combine=True, width=False)
    plt.savefig(f"Run 1 -- combined -- {g}.pdf")
    plt.close()

In [111]:
colourmap = plt.get_cmap("turbo")
data = np.array([noise_floor[s]["out"][1] for s in signals.keys()])
plt.bar(signals.keys(), data, color=colourmap(data/np.max(data)))
plt.xticks(rotation=90)
plt.ylabel("Noise floor (arbitrary units)")
plt.title("Noise floor of output signal at 9.2 Hz input")
plt.savefig("noise_floor_run_1_out.pdf",bbox_inches='tight')

In [32]:
colourmap = plt.get_cmap("turbo")
data = [peaks_w[s]["out"][1][0] for s in signals.keys()]
max_L = 0
print(data)

x_poses = np.arange(len(data))
w = 0.3
aa = ["Peak 1","Peak 2","Peak 3"]
data = list(zip(*data))
print(data)
for i, peeks in enumerate(data[:3]):
    plt.bar(x_poses + i*w - w/2, peeks, width=w,label=f"Peak {i}")

plt.xticks(x_poses, signals.keys(), rotation=90)
plt.legend()
plt.ylabel("Peak width (Hz)")
plt.title("Peak width of output signal at 9.2 Hz input")
plt.savefig("peak_width_run_1_out.pdf",bbox_inches='tight')

[[np.float64(0.10493565859396625), np.float64(0.10688930596254664), np.float64(0.1365717327486422)], [np.float64(0.13484189824525572), np.float64(0.13453196075718665), np.float64(0.16908821399625396)], [np.float64(0.24276817534180495), np.float64(0.2617634451373938), np.float64(0.2391508647761178)], [np.float64(0.46407058800627254), np.float64(0.4664966833051558), np.float64(0.5333922057761242)], [np.float64(0.2382211279806583), np.float64(0.23790369208759898), np.float64(0.2243737766206877)], [np.float64(0.10015546525065133), np.float64(0.10145179070246613), np.float64(0.1844270164185957)], [np.float64(0.11081266693299519)], [np.float64(0.20284636929429567), np.float64(0.22883742057367584), np.float64(0.16718022150801914)], [np.float64(0.2669922345334861), np.float64(0.2617707914590568), np.float64(0.27936937128459505)]]
[(np.float64(0.10493565859396625), np.float64(0.13484189824525572), np.float64(0.24276817534180495), np.float64(0.46407058800627254), np.float64(0.2382211279806583), 

In [24]:
colourmap = plt.get_cmap("turbo")

data = np.array([sorted(peaks[s]["out"][1][1])[-1] for s in signals.keys()])

plt.bar(signals.keys(), data, color=colourmap((data-np.min(data))/(np.max(data)-np.min(data))))
plt.xticks(rotation=90)
plt.ylabel("Main peak amplitude (arbitrary units)")
plt.title("Main peak amplitude of output signal at 9.2 Hz input")
plt.savefig("main_peak_amplitude_run_1_out.pdf",bbox_inches='tight')

## Run 2 (random)

In [27]:
for g in ["transfer"]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=2, combine=True, width=False)
    plt.savefig(f"Run 2 -- combined -- {g}.pdf")
    plt.close()

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:4: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])


In [113]:
colourmap = plt.get_cmap("turbo")
data = np.array([noise_floor[s]["transfer"][2] for s in signals.keys()])
plt.bar(signals.keys(), data, color=colourmap((data-np.min(data))/(np.max(data)-np.min(data))))
plt.xticks(rotation=90)
plt.ylabel("Intensity (f_ch2/f_ch1)")
plt.title("Transfer function noise floor inferred from random input")
plt.savefig("noise_floor_run_2_out.pdf",bbox_inches='tight')

In [35]:
colourmap = plt.get_cmap("turbo")
data = [peaks_w[s]["transfer"][2][0] for s in signals.keys()]
max_L = 0
print(data)

x_poses = np.arange(len(data))
w = 0.3
aa = ["Peak 1","Peak 2","Peak 3"]
data = list(zip(*data))
print(data)
for i, peeks in enumerate(data[:3]):
    plt.bar(x_poses + i*w - w/2, peeks, width=w,label=f"Peak {i}")

plt.xticks(x_poses, signals.keys(), rotation=90)
plt.legend()
plt.ylabel("Peak width (Hz)")
plt.title("Peak widths of transfer signal at random input")
plt.savefig("peak_width_transfer_run_2_out.pdf",bbox_inches='tight')

[[np.float64(0.31321489414835746), np.float64(0.24422800969787595), np.float64(0.10793290813704459)], [np.float64(0.10878086281555222), np.float64(0.15301946680820144), np.float64(0.5041961686849685)], [np.float64(0.1424056450438389), np.float64(0.23755679405296348), np.float64(0.10491465455485383)], [np.float64(1.121058985528407), np.float64(0.5997472282541718)], [np.float64(0.1076219704328163), np.float64(0.9696535261730936), np.float64(0.20935600213502426)], [np.float64(0.23682504665748816), np.float64(0.2629230266641347), np.float64(0.11852852798277524)], [np.float64(0.10884598918151411), np.float64(0.10822441479673373), np.float64(0.10859436491599439)], [np.float64(0.12453219705355378), np.float64(0.21905897844039757), np.float64(0.20521066107173702)], [np.float64(0.21027026906609514), np.float64(0.17400854838514057)]]
[(np.float64(0.31321489414835746), np.float64(0.10878086281555222), np.float64(0.1424056450438389), np.float64(1.121058985528407), np.float64(0.1076219704328163), n

In [54]:
colourmap = plt.get_cmap("turbo")

data = np.array([sorted(peaks[s]["transfer"][2][1])[-1] for s in signals.keys()])
plt.bar(signals.keys(), data, color=colourmap((data-np.min(data))/(15-np.min(data))))
plt.xticks(rotation=90)
plt.ylabel("Main peak amplitude (dB)")
plt.title("Main peak amplitude of transfer signal at random input")
plt.savefig("main_peak_amplitude_transfer_run_2_out.pdf",bbox_inches='tight')

## Run 3 (two resonant)

In [114]:
colourmap = plt.get_cmap("turbo")
data = np.array([noise_floor[s]["out"][3] for s in signals.keys()])
plt.bar(signals.keys(), data, color=colourmap(data*2/np.max(data)))
plt.xticks(rotation=90)
plt.ylabel("Noise floor (arbitrary units)")
plt.title("Noise floor of output signal at 9.2 Hz and 13.6 Hz input")
plt.savefig("noise_floor_run_3_out.pdf",bbox_inches='tight')

In [58]:
colourmap = plt.get_cmap("turbo")
data = [peaks_w[s]["out"][3][0] for s in signals.keys()]
max_L = 0
print(data)

x_poses = np.arange(len(data))
w = 0.3
aa = ["Peak 1","Peak 2","Peak 3"]
data = list(zip(*data))
print(data)
for i, peeks in enumerate(data[:2]):
    plt.bar(x_poses + i*w - w/2, peeks, width=w,label=f"Peak {i}")

plt.xticks(x_poses, signals.keys(), rotation=90)
plt.legend()
plt.ylabel("Peak width (Hz)")
plt.title("Peak width of output signal at 9.2 Hz and 13.6 Hz input")
plt.savefig("peak_width_run_3_out.pdf",bbox_inches='tight')

[[np.float64(0.10497396504619338), np.float64(0.1049170220954121), np.float64(0.10361862089432705)], [np.float64(0.13493665120570775), np.float64(0.1348658792892934), np.float64(0.13701559989365464)], [np.float64(0.24287394552881736), np.float64(0.24271599261083132), np.float64(0.24678898001208438)], [np.float64(0.46409783360441637), np.float64(0.46395809296225465), np.float64(0.47855830338960814)], [np.float64(0.2382341924024587), np.float64(0.23824387898989663), np.float64(0.24258778160100647)], [np.float64(0.10016587597275972), np.float64(0.10024318726137871), np.float64(0.10322195568569015)], [np.float64(0.1564662462391695), np.float64(0.11057349177823284), np.float64(0.0)], [np.float64(0.200686483493417), np.float64(0.20218195497773195), np.float64(0.1100903588578106)], [np.float64(0.26702447574965227), np.float64(0.2669660304682928), np.float64(0.28156006925735966)]]
[(np.float64(0.10497396504619338), np.float64(0.13493665120570775), np.float64(0.24287394552881736), np.float64(0.

In [62]:
colourmap = plt.get_cmap("turbo")

data = np.array([sorted(peaks[s]["out"][3][1])[-1] for s in signals.keys()])

plt.bar(signals.keys(), data, color=colourmap((data-np.min(data))/(np.max(data)-np.min(data))))
plt.xticks(rotation=90)
plt.ylabel("Main peak amplitude (arbitrary units)")
plt.title("Main peak amplitude of output signal at 9.2 Hz and 13.6 Hz input")
plt.savefig("main_peak_amplitude_run_3_out.pdf",bbox_inches='tight')

# Varying window length

In [185]:
key = "No windowing (2s)"
signals[key] = extract_data("code/myRect2Diary",length=2)
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,2):
        my_data = get_data(key, typ = L, run = r,plot=True,length=2)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

key = "No windowing (20s)"
signals[key] = extract_data("code/myRect20Diary",length=20)
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,2):
        my_data = get_data(key, typ = L, run = r,plot=True,length=20)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2633617901.py:7: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\creation.py:218: RuntimeWarning: invalid value encountered in linestrings
  return lib.linestrings(coords, np.intc(handle_nan), out=out, **kwargs)
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2921350288.py:4: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
C:\Users\gabri\AppData\Local\Temp\ipykernel_23032\2633617901.py:7: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\

In [119]:
key = "Flat-top (2s)"
signals[key] = extract_data("code/myFlattop2Diary",length=2)
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,2):
        my_data = get_data(key, typ = L, run = r,plot=True,length=2)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

key = "Flat-top (20s)"
signals[key] = extract_data("code/myFlattop20Diary",length=20)
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,2):
        my_data = get_data(key, typ = L, run = r,plot=True,length=2)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

In [186]:
colourmap = plt.get_cmap("turbo")
data_L = ["2s", "10s", "20s"]
data = [noise_floor[s]["out"][1] for s in ["Flat-top (2s)", "Flat-top", "Flat-top (20s)", "No windowing (2s)","No windowing","No windowing (20s)"]]
max_L = 0
print(data)

x_poses = np.arange(2)
w = 0.2
aa = ["Flat-top","No window"]
#data = list(zip(*data))
print(data)
for k in x_poses:
    for i, peeks in enumerate(data[k*3:k*3+3]):
        plt.bar(k + i*w - 1*w, peeks, width=w,label=f"{data_L[i%3]}" if k == 0 else None,color=f"C{i%3}")

plt.xticks(x_poses, aa)
plt.legend()
plt.ylabel("Noise (relative)")
plt.title("Noise floor of output at 9.2 Hz input")
plt.savefig("Noise_windows_pure.pdf")

[np.float64(0.7057997379758866), np.float64(0.9685277464563093), np.float64(0.6376471098358429), np.float64(0.49075541255472543), np.float64(0.9739585113602992), np.float64(0.6711358087412443)]
[np.float64(0.7057997379758866), np.float64(0.9685277464563093), np.float64(0.6376471098358429), np.float64(0.49075541255472543), np.float64(0.9739585113602992), np.float64(0.6711358087412443)]


In [187]:
colourmap = plt.get_cmap("turbo")
data_L = ["2s", "10s", "20s"]
data = [noise_floor[s]["transfer"][0] for s in ["Flat-top (2s)", "Flat-top", "Flat-top (20s)", "No windowing (2s)","No windowing","No windowing (20s)"]]
max_L = 0
print(data)

x_poses = np.arange(2)
w = 0.2
aa = ["Flat-top","No window"]
#data = list(zip(*data))
print(data)
for k in x_poses:
    for i, peeks in enumerate(data[k*3:k*3+3]):
        plt.bar(k + i*w - 1*w, peeks, width=w,label=f"{data_L[i%3]}" if k == 0 else None,color=f"C{i%3}")

plt.xticks(x_poses, aa)
plt.legend()
plt.ylabel("Noise (relative)")
plt.title("Noise floor of transfer at sine sweep input")
plt.savefig("Noise_windows_sweep.pdf")

[np.float64(0.028706942886851237), np.float64(0.03413515490085877), np.float64(0.8336822403189248), np.float64(0.06721227846636231), np.float64(0.060824755595626195), np.float64(0.0571386157608103)]
[np.float64(0.028706942886851237), np.float64(0.03413515490085877), np.float64(0.8336822403189248), np.float64(0.06721227846636231), np.float64(0.060824755595626195), np.float64(0.0571386157608103)]


In [159]:
colourmap = plt.get_cmap("turbo")
data_L = ["2s", "10s", "20s"]
data = [peaks_w[s]["out"][1][0][0] for s in ["Flat-top (2s)", "Flat-top", "Flat-top (20s)", "No windowing (2s)","No windowing","No windowing (20s)"]]
max_L = 0
print(data)

x_poses = np.arange(2)
w = 0.2
aa = ["Flat-top","No window"]
#data = list(zip(*data))
print(data)
for k in x_poses:
    for i, peeks in enumerate(data[k*3:k*3+3]):
        plt.bar(k + i*w - 1*w, peeks, width=w,label=f"{data_L[i%3]}" if k == 0 else None,color=f"C{i%3}")

plt.xticks(x_poses, aa)
plt.legend()
plt.ylabel("Peak width (Hz)")
plt.title("Peak width at 9.2 Hz input")
plt.savefig("windows-peak-width-1.pdf")

[np.float64(2.3062711130212534), np.float64(0.46407058800627254), np.float64(0.23236550762594632), np.float64(1.1721900365000604), np.float64(0.10015546525065133), np.float64(0.050169297131562374)]
[np.float64(2.3062711130212534), np.float64(0.46407058800627254), np.float64(0.23236550762594632), np.float64(1.1721900365000604), np.float64(0.10015546525065133), np.float64(0.050169297131562374)]


In [203]:
colourmap = plt.get_cmap("turbo")
data_L = ["2s", "10s", "20s"]
data = [sorted(peaks[s]["out"][1][1])[-1] for s in ["Flat-top (2s)", "Flat-top", "Flat-top (20s)", "No windowing (2s)","No windowing","No windowing (20s)"]]
max_L = 0
print(data)

x_poses = np.arange(2)
w = 0.2
aa = ["Flat-top","No window"]
#data = list(zip(*data))
print(data)
for k in x_poses:
    for i, peeks in enumerate(data[k*3:k*3+3]):
        plt.bar(k + i*w - 1*w, peeks/data[2 if k == 0 else 5], width=w,label=f"{data_L[i%3]}" if k == 0 else None,color=f"C{i%3}")

plt.xticks(x_poses, aa)
plt.legend()
plt.ylabel("Main peak amplitude (arbitrary units)")
plt.title("Main peak amplitude of output signal at 9.2Hz input")
plt.savefig("windows_peak_out_amplitude_1.pdf")

[np.float64(150.79121095077124), np.float64(633.3616908212873), np.float64(1028.9270458589374), np.float64(329.61647789514404), np.float64(2930.4873741410315), np.float64(4733.029558327308)]
[np.float64(150.79121095077124), np.float64(633.3616908212873), np.float64(1028.9270458589374), np.float64(329.61647789514404), np.float64(2930.4873741410315), np.float64(4733.029558327308)]


In [ ]:
colourmap = plt.get_cmap("turbo")
data_L = ["2s", "10s", "20s"]
data = [sorted(peaks[s]["transfer"][0][1])[-1] for s in ["Flat-top (2s)", "Flat-top", "Flat-top (20s)", "No windowing (2s)","No windowing","No windowing (20s)"]]
max_L = 0
print(data)

x_poses = np.arange(2)
w = 0.2
aa = ["Flat-top","No window"]
#data = list(zip(*data))
print(data)
for k in x_poses:
    for i, peeks in enumerate(data[k*3:k*3+3]):
        plt.bar(k + i*w - 1*w, peeks, width=w,label=f"{data_L[i%3]}" if k == 0 else None,color=f"C{i%3}")

plt.xticks(x_poses, aa)
plt.legend()
plt.ylabel("Main peak amplitude (dB)")
plt.title("Main peak amplitude of transfer signal at sweep input")
plt.savefig("windows_peak_transfer_amplitude_0.pdf")

[np.float64(2.9165466542086698), np.float64(24.188069795362825), np.float64(13.580073471797315), np.float64(21.072127243496652), np.float64(40.813580236785384), np.float64(26.029538286462333)]
[np.float64(2.9165466542086698), np.float64(24.188069795362825), np.float64(13.580073471797315), np.float64(21.072127243496652), np.float64(40.813580236785384), np.float64(26.029538286462333)]


In [194]:
colourmap = plt.get_cmap("turbo")
data_L = ["Flat-top (2s)", "Flat-top (10s)", "Flat-top (20s)", "No windowing (2s)","No windowing(10s)","No windowing (20s)"]
data = [list(peaks[s]["transfer"][0][0]) for s in ["Flat-top (2s)", "Flat-top", "Flat-top (20s)", "No windowing (2s)","No windowing","No windowing (20s)"]]
x_poses = np.arange(len(data[0]))
w = 0.08
aa = ["Peak 1", "Peak 2", "Peak 3"]
#data = list(zip(*data))
print(data)
for i, peeks in enumerate(data):
    plt.bar(x_poses + i*w - 2.5*w, peeks, width=w,label=f"{data_L[i]}")

plt.xticks(x_poses, aa, rotation=90)
plt.legend()
plt.ylabel("Peak place (Hz)")
plt.title("Largest 3 peaks of transfer at sweep input")
plt.savefig("window_peak_places_transfer_run_0_out.pdf",bbox_inches='tight')

[[0, 0, np.float64(14.5244)], [0, np.float64(9.0031), 0], [0, np.float64(9.3517), np.float64(13.4524)], [np.float64(3.005), np.float64(9.0151), np.float64(13.5227)], [np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [np.float64(3.2506), np.float64(9.2016), np.float64(13.4524)]]


In [191]:
colourmap = plt.get_cmap("turbo")
data_L = ["2s", "10s", "20s"]
data = [list(peaks[s]["out"][1][0])[1] for s in ["Flat-top (2s)", "Flat-top", "Flat-top (20s)", "No windowing (2s)","No windowing","No windowing (20s)"]]
print(peaks["Flat-top (2s)"]["out"][1][0])
x_poses = np.arange(2)
w = 0.2
aa = ["Flat-top","No window"]
#data = list(zip(*data))
print(data)
for k in x_poses:
    for i, peeks in enumerate(data[k*3:k*3+3]):
        plt.bar(k + i*w - 1*w, peeks, width=w,label=f"{data_L[i%3]}" if k == 0 else None,color=f"C{i%3}")

plt.xticks(x_poses, aa)
plt.yticks([2,4,6,8,10,9.2])
plt.legend()
plt.ylabel("Main peak position (Hz)")
plt.title("Main peak position of output signal at 9.2 Hz input")
plt.savefig("windows_peak_position_out_1.pdf")

[0, np.float64(9.516), 0]
[np.float64(9.516), np.float64(9.2032), np.float64(9.2016), np.float64(9.516), np.float64(9.2032), np.float64(9.2016)]
